In [1]:
# first neural network with keras tutorial
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

import numpy as np
import pandas as pd
import os
import PIL
import PIL.Image
import tensorflow as tf

#from sklearn.model_selection import train_test_split

In [2]:
train_ds = keras.utils.image_dataset_from_directory(
    'data/',
    validation_split=0.2,
    subset="training",
    image_size=(28, 28),
    seed=132,
    batch_size=32
)

Found 10000 files belonging to 10 classes.
Using 8000 files for training.


In [3]:
# let's separate train_ds into X_train and y_train

X_train = []
y_train = []

for images, labels in train_ds:
    for image in images:
        X_train.append(image)
    for label in labels:
        y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

In [4]:
val_ds = keras.utils.image_dataset_from_directory(
    'data/',
    validation_split=0.2,
    subset="validation",
    image_size=(28, 28),
    seed=132,
    batch_size=32
)

Found 10000 files belonging to 10 classes.
Using 2000 files for validation.


In [5]:
# checking class names

print(train_ds.class_names)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [6]:
# checking shape of data

for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 28, 28, 3)
(32,)


In [7]:
# import tensorflow as tf
# from tensorflow.keras.layers.experimental import preprocessing

# # Assume `x_train` and `y_train` are your training images and labels

# # Define the data augmentation
# data_augmentation = tf.keras.Sequential([
#   preprocessing.Rescaling(1./255),
#   preprocessing.RandomRotation(0.1),
#   preprocessing.RandomContrast(0.5),
# ])

# # Create empty lists to hold the augmented images and labels
# x_train_augmented = []
# y_train_augmented = []

# # For each image in the training set
# for x, y in zip(train_X, train_y):
#     # Create 5 augmented versions of the image
#     for _ in range(2):
#         augmented_image = data_augmentation(tf.expand_dims(x, 0), training=True)
#         x_train_augmented.append(tf.squeeze(augmented_image).numpy())
#         y_train_augmented.append(y)

# # Convert the lists to numpy arrays
# x_train_augmented = np.array(x_train_augmented)
# y_train_augmented = np.array(y_train_augmented)

In [8]:
# configure for performance

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [9]:
# standardize layer

normalization_layer = tf.keras.layers.Rescaling(1./255)

augmentation = tf.keras.Sequential([
    keras.layers.RandomBrightness(0.1),
    keras.layers.RandomRotation(0.1),
    keras.layers.RandomContrast(0.1),
])

In [10]:
# building our model

model = tf.keras.Sequential([
  # augmentation,
  normalization_layer,

  keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu'),
  keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2,2)),
  keras.layers.Dropout(0.2),

  keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu'),
  keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2,2)),
  keras.layers.Dropout(0.2),  

  tf.keras.layers.Flatten(),
  keras.layers.Dropout(0.2),

  tf.keras.layers.Dense(40, activation='relu'),
  tf.keras.layers.Dense(40, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

In [11]:
model.compile (
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [12]:
all_callbacks = keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    verbose=1,
    mode='auto',
    restore_best_weights=True
)

In [13]:
model.fit(
    train_ds,
    validation_data=val_ds,
    callbacks=[all_callbacks],
    epochs=100
)

Epoch 1/100
250/250 [==============================] - 2s 9ms/step - loss: 1.0499 - accuracy: 0.6284 - val_loss: 0.2557 - val_accuracy: 0.9200
Epoch 2/100
250/250 [==============================] - 2s 8ms/step - loss: 0.2598 - accuracy: 0.9183 - val_loss: 0.1715 - val_accuracy: 0.9455
Epoch 3/100
250/250 [==============================] - 2s 8ms/step - loss: 0.1828 - accuracy: 0.9417 - val_loss: 0.1322 - val_accuracy: 0.9585
Epoch 4/100
250/250 [==============================] - 2s 8ms/step - loss: 0.1527 - accuracy: 0.9485 - val_loss: 0.1270 - val_accuracy: 0.9610
Epoch 5/100
250/250 [==============================] - 2s 9ms/step - loss: 0.1248 - accuracy: 0.9581 - val_loss: 0.0968 - val_accuracy: 0.9675
Epoch 6/100
250/250 [==============================] - 2s 9ms/step - loss: 0.1019 - accuracy: 0.9649 - val_loss: 0.0884 - val_accuracy: 0.9740
Epoch 7/100
250/250 [==============================] - 2s 9ms/step - loss: 0.0978 - accuracy: 0.9674 - val_loss: 0.0936 - val_accuracy: 0.9705

In [14]:
score = model.evaluate(X_train, y_train, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.002760360948741436
Test accuracy: 0.9991250038146973


In [15]:
# VGG 16 - no standardization, no augmentation - 97.2% accuracy
# Somehow achieved 99.7% accuracy on test set, which i didn't expect, investigating...
# Now 99.91 which feels sorta impossible. Investigating...

In [16]:
# import matplotlib.pyplot as plt
# import numpy as np

# # Assume `data_augmentation` is your Sequential model containing the data augmentation layers
# # Assume `x_train` is your training data

# # Choose the first image from the training data
# image = train_X[0]

# # Apply the data augmentation to the image 5 times
# augmented_images = [data_augmentation(tf.expand_dims(image, 0), training=True) for _ in range(5)]

# # Remove the extra dimension added by tf.expand_dims
# augmented_images = [tf.squeeze(img).numpy() for img in augmented_images]

# # Plot the original image and the augmented images
# plt.figure(figsize=(10, 10))
# for i in range(6):
#     ax = plt.subplot(3, 2, i + 1)
#     if i == 0:
#         plt.imshow(image)
#         plt.title("Original image")
#     else:
#         plt.imshow(augmented_images[i - 1])
#         plt.title(f"Augmented image {i}")
#     plt.axis("off")